Import required packages for model

In [130]:
## os to get audio file paths
import os
## re to replace filenames
import re
## data cleaning, normalize strange chars
#import unicode

import pathlib
import numpy as np
import librosa
import librosa.display
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models

from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

In [132]:
DATASET_PATH = r"C:\Users\shane\Documents\GitHub\Chord_App\audio_data\data"
train_path = r"C:\Users\shane\Documents\GitHub\Chord_App\audio_data\data\Training"
test_path = r"C:\Users\shane\Documents\GitHub\Chord_App\audio_data\data\Test"


#x_test, y_test, test_labels = load_audio_files(test_path)

In [133]:
train_data_dir = pathlib.Path(train_path)
test_data_dir = pathlib.Path(test_path)
if not train_data_dir.exists():
    print("Training path does not exist. Please check dataset location.")

if not test_data_dir.exists():
    print("Test path does not exist. Please check dataset location.")


In [134]:
# labels for each dataset of chords
training_folder_names = np.array(tf.io.gfile.listdir(str(train_data_dir)))
print("training_folder_names: ", training_folder_names)

testing_folder_names = np.array(tf.io.gfile.listdir(str(test_data_dir)))
print("test_folder_names: ", test_folder_names)

base_path = r"C:\Users\shane\Documents\GitHub\Chord_App\audio_data\data\Training"
training_folder_paths = []
testing_folder_paths = []

#Create array of chord paths in the training data folder.
training_folder_paths = [os.path.join(base_path, folderName) for folderName in training_folder_names if os.path.isdir(os.path.join(base_path, folderName))]
testing_folder_paths = [os.path.join(base_path, folderName) for folderName in testing_folder_names if os.path.isdir(os.path.join(base_path, folderName))]

print("Training paths:")
for path in training_folder_paths:
    print(path + "\n")

print("Testing paths:")
for path in testing_folder_paths:
    print(path + "\n")

training_folder_names:  ['Am' 'Bb' 'Bdim' 'C' 'Dm' 'Em' 'F' 'G']
test_folder_names:  ['Am' 'Bb' 'Bdim' 'C' 'Dm' 'Em' 'F' 'G']
Training paths:
C:\Users\shane\Documents\GitHub\Chord_App\audio_data\data\Training\Am

C:\Users\shane\Documents\GitHub\Chord_App\audio_data\data\Training\Bb

C:\Users\shane\Documents\GitHub\Chord_App\audio_data\data\Training\Bdim

C:\Users\shane\Documents\GitHub\Chord_App\audio_data\data\Training\C

C:\Users\shane\Documents\GitHub\Chord_App\audio_data\data\Training\Dm

C:\Users\shane\Documents\GitHub\Chord_App\audio_data\data\Training\Em

C:\Users\shane\Documents\GitHub\Chord_App\audio_data\data\Training\F

C:\Users\shane\Documents\GitHub\Chord_App\audio_data\data\Training\G

Testing paths:
C:\Users\shane\Documents\GitHub\Chord_App\audio_data\data\Training\Am

C:\Users\shane\Documents\GitHub\Chord_App\audio_data\data\Training\Bb

C:\Users\shane\Documents\GitHub\Chord_App\audio_data\data\Training\Bdim

C:\Users\shane\Documents\GitHub\Chord_App\audio_data\data\Tra

In [207]:
print("Training samples: " + "\n")
for root, dirs, files in os.walk(train_path):
    for name in files:
        print(name)
training_files = []
print("Testing samples: " + "\n")
for root, dirs, files in os.walk(test_path):
    for name in files:
        training_files.append(name)
        print(name)

Training samples: 

Am_acousticguitar_Mari_1.wav
Am_acousticguitar_Mari_2.wav
Am_acousticguitar_Mari_3.wav
Am_acousticguitar_Mari_4.wav
Am_acousticguitar_SalomCΓòá-Andrade_1.wav
Am_acousticguitar_SalomCΓòá-Andrade_2.wav
Am_acousticguitar_SalomCΓòá-Andrade_3.wav
Am_acousticguitar_SalomCΓòá-Andrade_4.wav
Am_acusticGB_JO_1.wav
Am_acusticGB_JO_2.wav
Am_acusticGB_JO_3.wav
Am_acusticGB_JO_4.wav
Am_AcusticPlug15_1.wav
Am_AcusticPlug15_2.wav
Am_AcusticPlug15_3.wav
Am_AcusticPlug15_4.wav
Am_AcusticPlug16_1.wav
Am_AcusticPlug16_2.wav
Am_AcusticPlug16_3.wav
Am_AcusticPlug16_4.wav
Am_AcusticPlug17_1.wav
Am_AcusticPlug17_2.wav
Am_AcusticPlug17_3.wav
Am_AcusticPlug17_4.wav
Am_AcusticPlug18_1.wav
Am_AcusticPlug18_2.wav
Am_AcusticPlug18_3.wav
Am_AcusticPlug18_4.wav
Am_AcusticPlug19_1.wav
Am_AcusticPlug19_2.wav
Am_AcusticPlug19_3.wav
Am_AcusticPlug19_4.wav
Am_AcusticPlug1_1.wav
Am_AcusticPlug1_2.wav
Am_AcusticPlug1_3.wav
Am_AcusticPlug1_4.wav
Am_AcusticPlug20_1.wav
Am_AcusticPlug20_2.wav
Am_AcusticPlug

In [210]:
# data hygene before processing: Filenames with weird characters

def clean_filename(filename):
    name, ext = os.path.splitext(filename)
    clean_name = re.sub(r'[^\w\d-]+','-', name)
    return clean_name + ext
    
def clean_folder(folder_path):
    for fname in os.listdir(folder_path):
        old_path = os.path.join(folder_path, fname)
        if os.path.isfile(old_path):
            new_fname = clean_filename(fname)
            new_path = os.path.join(folder_path, new_fname)
            if old_path != new_path:
                #print(f"Renaiming:\n {old_path} to {new_path}")
                os.rename(old_path, new_path)


training_base_path = r"C:\Users\shane\Documents\GitHub\Chord_App\audio_data\data\Training"
testing_base_path = r"C:\Users\shane\Documents\GitHub\Chord_App\audio_data\data\Test"

######### Clean Training data

train_folder_names = np.array(tf.io.gfile.listdir(str(train_data_dir)))
folder_paths = []

#Create array of chord paths in the training data folder.
training_folder_paths = [os.path.join(training_base_path, folderName) for folderName in train_folder_names if os.path.isdir(os.path.join(training_base_path, folderName))]


#Loop all paths, clean file names in the paths
for paths in training_folder_paths:
    clean_folder(paths)


for path in training_folder_paths:
    print("Cleaned: " + path + "\n")
    for root, dirs, files in os.walk(path):
        for name in files:
            print(name)
        print("\n")

######### Clean Testing data

test_folder_names = np.array(tf.io.gfile.listdir(str(train_data_dir)))
folder_paths = []
        
#Create array of chord paths in the testing data folder.
testing_folder_paths = [os.path.join(testing_base_path, folderName) for folderName in test_folder_names if os.path.isdir(os.path.join(testing_base_path, folderName))]

#Loop all paths, clean file names in the paths
for path in testing_folder_paths:
    clean_folder(path)


for paths in testing_folder_paths:
    print("Cleaned: " + paths + "\n")
    for root, dirs, files in os.walk(paths):
        for name in files:
            print(name)
        print("\n")
 


Cleaned: C:\Users\shane\Documents\GitHub\Chord_App\audio_data\data\Training\Am

Am_acousticguitar_Mari_1.wav
Am_acousticguitar_Mari_2.wav
Am_acousticguitar_Mari_3.wav
Am_acousticguitar_Mari_4.wav
Am_acousticguitar_SalomCΓòá-Andrade_1.wav
Am_acousticguitar_SalomCΓòá-Andrade_2.wav
Am_acousticguitar_SalomCΓòá-Andrade_3.wav
Am_acousticguitar_SalomCΓòá-Andrade_4.wav
Am_acusticGB_JO_1.wav
Am_acusticGB_JO_2.wav
Am_acusticGB_JO_3.wav
Am_acusticGB_JO_4.wav
Am_AcusticPlug15_1.wav
Am_AcusticPlug15_2.wav
Am_AcusticPlug15_3.wav
Am_AcusticPlug15_4.wav
Am_AcusticPlug16_1.wav
Am_AcusticPlug16_2.wav
Am_AcusticPlug16_3.wav
Am_AcusticPlug16_4.wav
Am_AcusticPlug17_1.wav
Am_AcusticPlug17_2.wav
Am_AcusticPlug17_3.wav
Am_AcusticPlug17_4.wav
Am_AcusticPlug18_1.wav
Am_AcusticPlug18_2.wav
Am_AcusticPlug18_3.wav
Am_AcusticPlug18_4.wav
Am_AcusticPlug19_1.wav
Am_AcusticPlug19_2.wav
Am_AcusticPlug19_3.wav
Am_AcusticPlug19_4.wav
Am_AcusticPlug1_1.wav
Am_AcusticPlug1_2.wav
Am_AcusticPlug1_3.wav
Am_AcusticPlug1_4.wav


In [211]:
totalCount = 0
for path in training_folder_paths:
    count = 0
    for root, dirs, files in os.walk(path):
        for name in files:
            count += 1
    print(path, " total: ", count)
    totalCount += count


for root, dirs, files in os.walk(train_path):
    for name in files:
        print(name)


print( "\n", "Training file size: ", totalCount, "\n")
totalCount = 0
for path in testing_folder_paths:
    count = 0
    for root, dirs, files in os.walk(path):
        for name in files:
            count += 1
    print(path, " total: ", count)
    totalCount += count
    

print( "\n", "Testing file size: ", totalCount)

C:\Users\shane\Documents\GitHub\Chord_App\audio_data\data\Training\Am  total:  180
C:\Users\shane\Documents\GitHub\Chord_App\audio_data\data\Training\Bb  total:  180
C:\Users\shane\Documents\GitHub\Chord_App\audio_data\data\Training\Bdim  total:  180
C:\Users\shane\Documents\GitHub\Chord_App\audio_data\data\Training\C  total:  180
C:\Users\shane\Documents\GitHub\Chord_App\audio_data\data\Training\Dm  total:  180
C:\Users\shane\Documents\GitHub\Chord_App\audio_data\data\Training\Em  total:  180
C:\Users\shane\Documents\GitHub\Chord_App\audio_data\data\Training\F  total:  180
C:\Users\shane\Documents\GitHub\Chord_App\audio_data\data\Training\G  total:  180
Am_acousticguitar_Mari_1.wav
Am_acousticguitar_Mari_2.wav
Am_acousticguitar_Mari_3.wav
Am_acousticguitar_Mari_4.wav
Am_acousticguitar_SalomCΓòá-Andrade_1.wav
Am_acousticguitar_SalomCΓòá-Andrade_2.wav
Am_acousticguitar_SalomCΓòá-Andrade_3.wav
Am_acousticguitar_SalomCΓòá-Andrade_4.wav
Am_acusticGB_JO_1.wav
Am_acusticGB_JO_2.wav
Am_acusti

In [212]:
test_files = [] 

for path in testing_folder_paths:
    for root, dirs, files in os.walk(path):
        for testfile in files:
             test_files.append(os.path.join(root,testfile))           
                
print("Testable files per folder: ", len(files))
print("Testable files: ", len(test_files))

test_file = tf.io.read_file(np.random.choice(test_files))

test_audio, sample_rate = tf.audio.decode_wav(contents=test_file, desired_channels=1)
print("Audio Shape: ", test_audio.shape)
print("Audio Rate: ", sample_rate)

Testable files per folder:  40
Testable files:  320
Audio Shape:  (102028, 1)
Audio Rate:  tf.Tensor(16000, shape=(), dtype=int32)


Decode file to create a waveform image

In [224]:
audio = tf.squeeze(test_audio, axis=-1)
stft = tf.signal.stft(
    signals=audio,
    frame_length=1024,
    frame_step=256,
    fft_length=1024
)
spectrogram = tf.abs(stft)

print(stft.shape)
print(spectrogram.shape)

(395, 513)
(395, 513)
